In [1]:
import import_ipynb
import numpy as np
import math

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from tools.Analysis_Tools import *

In [1]:
def print_a_antero_posterior_result(a_centriole_list, which_antero_posterior_segment, n_ante_post_segment = 5, a_lat_size = 0.1, a_lat_step = 0.05, save = False, path = None):
    ''' Function that print on a graph the angle of every centriole that belong to the 'which..' antero-posterior segment.
    In addition the medioLateral moving average is computed as well as circular std.
    
    Arguments:
        -> a_centriole_list: a list of reoriented centriole generated with the appropriate function. Format: 
                                                    [ Idx, 
                                                    
                                                    Relative lateral distance,
                                                    Relative longitudinal distance,
                                                    Angle]
        
        -> which_antero_posterior_segment: the segment of interest 
        -> n_ante_post_segment: the number of antero-posterior segment wanted
        -> a_lat_size: the size of medio-lateral segmentation to compute moving average (min>0, max = 1)
        -> a_lat_step: size of the step to compute the moving average
    
    '''
    
    moving_average, moving_std = moving_average_angle(a_centriole_list, 
                     a_lateral_size = a_lat_size, 
                     lat_step = a_lat_step, 
                     a_longitudinal_size = 1/n_ante_post_segment, 
                     long_step = 1/n_ante_post_segment)
    
    y_ma_values = moving_average[which_antero_posterior_segment]
    y_cstd_values = moving_std[which_antero_posterior_segment]
    
    x_ma_values = np.linspace(a_lat_size/2, 1-a_lat_size/2, len(y_ma_values))
    
    longitudinal_ = which_antero_posterior_segment + 1
       
    plt.figure(figsize=(15,8))
    
    plt.plot(x_ma_values, y_ma_values, color = 'k')
    plt.fill_between(x_ma_values, (y_ma_values-y_cstd_values), (y_ma_values+y_cstd_values), color = 'whitesmoke')
                     
    color_graph = ['orange', 'green', 'purple', 'cyan', 'blue']
    for a_centriole in a_centriole_list:
        if a_centriole[5] == 'left':
            lat = abs(a_centriole[6]-1)
        else: 
            lat = a_centriole[6] + 1
        
        if int(a_centriole[7]//(1/n_ante_post_segment) +1) == longitudinal_:
            plt.plot(lat/2, a_centriole[8], color = color_graph[which_antero_posterior_segment], marker = 'x', markersize = 1)
    

    plt.title('Individual centriole angle')
    plt.xlabel('Left side                   ||                  Right side')
    plt.ylabel('Centriole Angle')
    
    plt.ylim(-180, 180)
    plt.xlim(0,1)
    
    plt.xticks(np.arange(0, 1.01, step = 0.1))
    plt.yticks(np.arange(-180, 181, step=30))
    
    plt.grid(axis ='y', linestyle='--', linewidth=0.5)
    plt.vlines(x = 0.5, ymin = -180, ymax = 180,  linestyle='--', linewidth=0.5, color ='r')
    
    if save:
        newPath = path + '_GRAPH_' + which_antero_posterior_segment + '.tif'
        plt.savefig(newPath)
        
        newPath = path + '_GRAPH_DATA_MEAN_' + which_antero_posterior_segment + '.csv'
        with open(newPath, 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(moving_average)
            
        newPath = path + '_GRAPH_DATA_CSTD_' + which_antero_posterior_segment + '.csv'
        with open(newPath, 'w', newline='') as myfile:
            wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
            wr.writerow(moving_std)
        
    
    
    plt.show()

In [4]:
# Reformating of the centriole coordinates for graphical purpose
def Worm_And_Centriole(a_list_of_centriole, a_worm,  a_list_of_desired_centriole, coords, save = False, path = None):
    """ A function that draw the edge and midline of a worm + the localisation of all centrioles in a list.
    In addition, some specific centrioles can be highlighted as well as soecific X/Y coordinates
    
    Arguments:
        -> a_list_of_centriole: Should fit the format: [ Idx, 
                                                    
                                                    Relative lateral distance,
                                                    Relative longitudinal distance,
                                                    Angle]
        -> a_worm: midline and edge x/Y coordinates of a worm. Format: [x_midline, y_midline, x_edge, y_edge]
        -> a_list_of_desired_centriole: a list of integer corresponding to centrioles idx that you want to see
        -> coords: a tuple of X/Y coordinate
        -> path : a path where the figure will be save
        -> save : Do you want to save the figure (Bool)"""

    x_mid, y_mid, x_edg, y_edg = a_worm[0], a_worm[1], a_worm[2], a_worm[3]
    
    
    colors_ = ['darkorange', 'darkviolet', 'dodgerblue', 'crimson', 'magenta' ]
    markers_ = ['o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X']


    plt.figure(figsize=(50,20))

    # Plot Worm edge and midline
    plt.plot(x_mid, -y_mid)
    plt.plot(x_edg, -y_edg)

    # Plot All centrioles
    for z in a_list_of_centriole:
        if z[5] == 'left':
            plt.plot(z[1], z[2], 'bo', markersize = 1)
        else:
            plt.plot(z[1], z[2], 'ro', markersize = 1)

    # Plot centriole of interests
    color_idx = 0
    marker_idx = 0
    if len(a_list_of_desired_centriole) !=0:
        print(' {:^4}|{:^7}|{:^7}|{:^7}  ||{:^8}| {:<}'.format('Idx', 'Angle', 'Lateral', 'Longi', 'Marker',  'Color'))
        print('------------------------------------------------')

        for i in a_list_of_desired_centriole:
            plt.plot(a_list_of_centriole[i][1], a_list_of_centriole[i][2], color = colors_[color_idx], marker = markers_[marker_idx], markersize = 10)
            print('{:>4} |{:5.0f}  |{:6.2f} |{:6.2f}   ||{:^8s}| {}'.format(i, a_list_of_centriole[i][8], a_list_of_centriole[i][6], a_list_of_centriole[i][7], markers_[marker_idx], colors_[color_idx]))
            if color_idx == (len(colors_)-1):
                color_idx = 0
                marker_idx +=1
                if marker_idx == len(markers_):
                    marker_idx = 0
            else:
                color_idx +=1


    # Plot a specific coordinate
    if coords[0] == None: 
        pass
    else:
        plt.plot(coords[0], coords[1], color = 'darkviolet', marker = 'x', markersize = 10)

    plt.title('This is a worm')
    plt.xlabel('xAxis name')
    plt.ylabel('yAxis name')
    
    if save:
        newPath = path + '_Centrioles_Localization_.tif'
        plt.savefig(newPath)
        
    plt.show()